<img src="https://github.com/chdb-io/chdb/raw/main/docs/_static/snake-chdb.png" height=100>

# chDB Colab

Compare chDB / ClickHouse and Pandas performance with .Parquet files

## Setup
First we download some files and install the required libraries.

In [ ]:
!pip install pyarrow pandas
!pip install chdb --pre --upgrade

!mkdir -p taxi
!wget https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet -O taxi/201904.parquet
!wget https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_05.parquet -O taxi/201905.parquet
!wget https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_06.parquet -O taxi/201906.parquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 MB 6.1 MB/s eta 0:00:00
--2023-06-15 20:34:08--  https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/duckdb/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet [following]
--2023-06-15 20:34:09--  https://github.com/duckdb/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/263853960/d481ee00-d5d4-11eb-85f7-c0e2

In [ ]:
import pyarrow.parquet as pq
import pandas
import glob
import chdb

# no setup for chDB
chdb.query("SELECT version()", 'Dataframe')

,version()
0,b'22.12.1.1'


## Reading Multiple Parquet Files

chDB can read multiple parquet files using the glob syntax.

In Pandas, we need to load the files separately and concatenate them together into a single DataFrame.

In [ ]:
%%time
chdb.query('SELECT * FROM file("taxi/*.parquet", Parquet) LIMIT 5', 'Dataframe');


CPU times: user 57.9 ms, sys: 20.1 ms, total: 78 ms
Wall time: 80.8 ms


,vendor_id,pickup_at,dropoff_at,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,b'1',2019-04-01 00:04:09+00:00,2019-04-01 00:06:35+00:00,1,0.5,b'1',b'N',239,239,b'1',4.0,3.0,0.5,1.00,0.0,0.3,8.80,2.5
1,b'1',2019-04-01 00:22:45+00:00,2019-04-01 00:25:43+00:00,1,0.7,b'1',b'N',230,100,b'2',4.5,3.0,0.5,0.00,0.0,0.3,8.30,2.5
2,b'1',2019-04-01 00:39:48+00:00,2019-04-01 01:19:39+00:00,1,10.9,b'1',b'N',68,127,b'1',36.0,3.0,0.5,7.95,0.0,0.3,47.75,2.5
3,b'1',2019-04-01 00:35:32+00:00,2019-04-01 00:37:11+00:00,1,0.2,b'1',b'N',68,68,b'2',3.5,3.0,0.5,0.00,0.0,0.3,7.30,2.5
4,b'1',2019-04-01 00:44:05+00:00,2019-04-01 00:57:58+00:00,1,4.8,b'1',b'N',50,42,b'1',15.5,3.0,0.5,3.85,0.0,0.3,23.15,2.5


In [ ]:
%%time
df = pandas.concat(
	[pandas.read_parquet(file)
	 for file
	 in glob.glob('taxi/*.parquet')])
print(df.head(5))

  vendor_id           pickup_at          dropoff_at  passenger_count  \
0         1 2019-06-01 00:55:13 2019-06-01 00:56:17                1   
1         1 2019-06-01 00:06:31 2019-06-01 00:06:52                1   
2         1 2019-06-01 00:17:05 2019-06-01 00:36:38                1   
3         1 2019-06-01 00:59:02 2019-06-01 00:59:12                0   
4         1 2019-06-01 00:03:25 2019-06-01 00:15:42                1   

   trip_distance rate_code_id store_and_fwd_flag  pickup_location_id  \
0            0.0            1                  N                 145   
1            0.0            1                  N                 262   
2            4.4            1                  N                  74   
3            0.8            1                  N                 145   
4            1.7            1                  N                 113   

   dropoff_location_id payment_type  fare_amount  extra  mta_tax  tip_amount  \
0                  145            2          3.0    0.

## Concatenate the three files into a single large file

As Pandas does not have native support for reading multiple files, we perform the remaining experiments on a single large file.

We use the pyarrow library to concatenate the three files into a single file.

In [ ]:
# concatenate all three parquet files into a single file
pq.write_table(pq.ParquetDataset('taxi/').read(), 'alltaxi.parquet', row_group_size=100000)

### Querying the Single File

Now let's query the single file and check the achieved performance.

In [ ]:
%%time
# chDB
chdb.query('SELECT * FROM file("alltaxi.parquet", Parquet) LIMIT 5', 'Dataframe');


CPU times: user 57.2 ms, sys: 12.3 ms, total: 69.5 ms
Wall time: 71.6 ms


,vendor_id,pickup_at,dropoff_at,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,b'1',2019-04-01 00:04:09+00:00,2019-04-01 00:06:35+00:00,1,0.5,b'1',b'N',239,239,b'1',4.0,3.0,0.5,1.00,0.0,0.3,8.80,2.5
1,b'1',2019-04-01 00:22:45+00:00,2019-04-01 00:25:43+00:00,1,0.7,b'1',b'N',230,100,b'2',4.5,3.0,0.5,0.00,0.0,0.3,8.30,2.5
2,b'1',2019-04-01 00:39:48+00:00,2019-04-01 01:19:39+00:00,1,10.9,b'1',b'N',68,127,b'1',36.0,3.0,0.5,7.95,0.0,0.3,47.75,2.5
3,b'1',2019-04-01 00:35:32+00:00,2019-04-01 00:37:11+00:00,1,0.2,b'1',b'N',68,68,b'2',3.5,3.0,0.5,0.00,0.0,0.3,7.30,2.5
4,b'1',2019-04-01 00:44:05+00:00,2019-04-01 00:57:58+00:00,1,4.8,b'1',b'N',50,42,b'1',15.5,3.0,0.5,3.85,0.0,0.3,23.15,2.5


In [ ]:
%%time
pandas.read_parquet('alltaxi.parquet').head(5)

CPU times: user 12.3 s, sys: 4.23 s, total: 16.5 s
Wall time: 10.8 s


,vendor_id,pickup_at,dropoff_at,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-04-01 00:04:09,2019-04-01 00:06:35,1,0.5,1,N,239,239,1,4.0,3.0,0.5,1.00,0.0,0.3,8.80,2.5
1,1,2019-04-01 00:22:45,2019-04-01 00:25:43,1,0.7,1,N,230,100,2,4.5,3.0,0.5,0.00,0.0,0.3,8.30,2.5
2,1,2019-04-01 00:39:48,2019-04-01 01:19:39,1,10.9,1,N,68,127,1,36.0,3.0,0.5,7.95,0.0,0.3,47.75,2.5
3,1,2019-04-01 00:35:32,2019-04-01 00:37:11,1,0.2,1,N,68,68,2,3.5,3.0,0.5,0.00,0.0,0.3,7.30,2.5
4,1,2019-04-01 00:44:05,2019-04-01 00:57:58,1,4.8,1,N,50,42,1,15.5,3.0,0.5,3.85,0.0,0.3,23.15,2.5


## Counting the Rows

Now suppose we want to figure out how many rows are in our data set. We can do that using the following code snippets.

Note that by default Pandas will read the entire Parquet file into memory again. We can manually optimize the query by specifying that only a single column should be loaded.



In [ ]:
%%time
# chDB
chdb.query('SELECT count(*) FROM file("alltaxi.parquet", Parquet) LIMIT 5', 'Dataframe');


CPU times: user 182 ms, sys: 83 ms, total: 265 ms
Wall time: 255 ms


,count()
0,21939424


In [ ]:
%%time
# Pandas (naive)
print(len(pandas.read_parquet('alltaxi.parquet')))

21939424
CPU times: user 11.4 s, sys: 5.53 s, total: 16.9 s
Wall time: 11.3 s


In [ ]:
%%time
# Pandas (projection pushdown)
print(len(pandas.read_parquet('alltaxi.parquet', columns=['vendor_id'])))

21939424
CPU times: user 1.18 s, sys: 473 ms, total: 1.65 s
Wall time: 1e+03 ms


# Filtering Rows
It is common to use some sort of filtering predicate to only look at the interesting parts of a data set. For example, imagine we want to know how many taxi rides occur after the 30th of June 2019. We can do that using the following queries in both Pandas and chDB.


In [ ]:
%%time
# chDB
chdb.query("SELECT count(*) FROM file('alltaxi.parquet', Parquet)  WHERE pickup_at > '2019-06-30'", 'Dataframe');

CPU times: user 386 ms, sys: 156 ms, total: 543 ms
Wall time: 515 ms


,count()
0,167022


In [ ]:
%%time
# Pandas (naive)
len(pandas.read_parquet('alltaxi.parquet')
          .query("pickup_at > '2019-06-30'"))

CPU times: user 8.72 s, sys: 3.88 s, total: 12.6 s
Wall time: 8.64 s


167022

In [ ]:
%%time
# Pandas (projection pushdown)
len(pandas.read_parquet('alltaxi.parquet', columns=['pickup_at'])
          .query("pickup_at > '2019-06-30'"))

CPU times: user 758 ms, sys: 670 ms, total: 1.43 s
Wall time: 1.11 s


167022

In [ ]:
df = pandas.read_parquet('alltaxi.parquet')

In [ ]:
%%time
# Pandas native
print(len(df[['pickup_at']].query("pickup_at > '2019-06-30'")))

167022
CPU times: user 547 ms, sys: 556 ms, total: 1.1 s
Wall time: 883 ms


## Aggregates

Now suppose we want to figure out how many rows are in our data set. We can do that using the following code snippets.

Note that by default Pandas will read the entire Parquet file into memory again. We can manually optimize the query by specifying that only a single column should be loaded.



In [ ]:
%%time
# chDB (SQL)
chdb.query("SELECT passenger_count, count(*) FROM file('alltaxi.parquet', Parquet) GROUP BY passenger_count", 'Dataframe');

CPU times: user 839 ms, sys: 185 ms, total: 1.02 s
Wall time: 1.07 s


,passenger_count,count()
0,0,408742
1,7,106
2,1,15356631
3,6,546467
4,9,64
5,2,3332927
6,5,910516
7,8,72
8,3,944833
9,4,439066


In [ ]:
%%time
# Pandas (naive)
pandas.read_parquet('alltaxi.parquet').groupby('passenger_count').agg({'passenger_count' : 'count'})

CPU times: user 8.67 s, sys: 4.34 s, total: 13 s
Wall time: 8.37 s


,passenger_count
passenger_count,
0,408742
1,15356631
2,3332927
3,944833
4,439066
5,910516
6,546467
7,106
8,72


In [ ]:
%%time
# Pandas (projection pushdown)
pandas.read_parquet('alltaxi.parquet', columns=['passenger_count']).groupby('passenger_count').agg({'passenger_count' : 'count'})

CPU times: user 533 ms, sys: 179 ms, total: 712 ms
Wall time: 529 ms


,passenger_count
passenger_count,
0,408742
1,15356631
2,3332927
3,944833
4,439066
5,910516
6,546467
7,106
8,72


In [ ]:
%%time
# Pandas (native)
df[['passenger_count']].groupby('passenger_count').agg({'passenger_count' : 'count'})

CPU times: user 334 ms, sys: 87.8 ms, total: 421 ms
Wall time: 547 ms


,passenger_count
passenger_count,
0,408742
1,15356631
2,3332927
3,944833
4,439066
5,910516
6,546467
7,106
8,72
